<a href="https://colab.research.google.com/github/yip-i/asr_capstone/blob/main/xls_r_53_fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # **Fine-tuning Wav2Vec2 for Torgo DataSet with 🤗 Transformers**

# **Ensure that GPU and RAM is set up: will be needed for training purpose**

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 11 22:56:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    33W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# ensure enough memory present so that training does not stop
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# **Packages needed:** <br>
`datasets`: to transform the dataset <br>
`transformers`: upgraded version of RNN (allows to process a large quantity of text) <br>
`librosa`: needed for the audio files <br>
`jiwer`: **most important:** WER metric

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.23.1
!pip install jiwer
!pip install librosa
# %cd /content/espnet/tools
!make CUDA_VERSION=10.2

# **Download the torgo dataset that will be finetuned against the state of the art model**

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
%cd /content
# !gdown 1FUls9tWqAPD9mggqzkaDtIhNYE0fJ1aF
!gdown 1TaP_UX2QiiKR_qWMeYtlMgHaXlYRliJ9


/content
Downloading...
From: https://drive.google.com/uc?id=1TaP_UX2QiiKR_qWMeYtlMgHaXlYRliJ9
To: /content/output.csv
100% 607k/607k [00:00<00:00, 141MB/s]


In [ ]:
!pip install --upgrade --no-cache-dir gdown


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:

!mkdir downloads
%cd downloads
# Original Link
!gdown 1hu3l5E8OY8jMHSIN2Bafg3cMw-icmlZM
# !gdown 1Mlepu0-Oxg2YorRngtWcvIX0jgp-q7iy
!tar -xzf torgo.tar.gz && ls torgo


Streaming output truncated to the last 5000 lines.
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar

# **Connect to Hugging Face to store the results of the model**

In [ ]:
# to store the model checkpoints, we will need to install another package
%%capture
!apt install git-lfs

# !!! Important: Run the above before everything

#                        .

#                            ..

# **Crucial stage: Preparation of data, Tokenizer and Feature Extractor**

ASR models transcribe speech to text which leads to the requirement of a feature extractor and tokenizer: <br>
`feature extractor`: processes speech signal to the required input format: audio processing: feature vector <br>
`tokenizer`: converts the model's output to text format <br>
`wave2vec2` has the following tokenizer: `wave2vec2CTCTokenizer` and feature extractor: `wave2vec2FeatureExtractor`

# **Tokenizer**

In [ ]:
# load the dataset, observe structure, divide into training and test set (evaluation later)
from datasets import load_dataset, load_metric, DatasetDict, Dataset, Audio

data = load_dataset('csv', data_files='/content/output.csv')
print(data)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-8772b7a42ca8c4c7/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'session', 'text', 'audio', 'speaker_id'],
        num_rows: 5534
    })
})


In [ ]:
speaker = "F04"
repo_name = "wav2vec2-demo-{}-2".format(speaker)


In [ ]:
repo_name

'wav2vec2-demo-F04-2'

In [ ]:
# creating a train and testing dataset
torgo_dataset = DatasetDict()

torgo_dataset['train'] = data['train'].filter(lambda x: x != speaker, input_columns=['speaker_id'])
torgo_dataset['test'] = data['train'].filter(lambda x: x == speaker, input_columns=['speaker_id'])

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [ ]:
# remove columns that we do not need
torgo_dataset = torgo_dataset.remove_columns(["Unnamed: 0"])
torgo_dataset

DatasetDict({
    train: Dataset({
        features: ['session', 'text', 'audio', 'speaker_id'],
        num_rows: 4867
    })
    test: Dataset({
        features: ['session', 'text', 'audio', 'speaker_id'],
        num_rows: 667
    })
})

In [ ]:
# ignore special characters: with no language model hard to classify them
# also convert all the text into lowercase: makes life much more easier
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
# use map function to carry out the process/transformation
torgo_dataset = torgo_dataset.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
# write a function that will first concatenate all the transcriptions to one single transcription and them we map them to characters
# In short: creating tokens: determine the length of array

def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = torgo_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=torgo_dataset.column_names["train"])


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into 
# an enumerated dictionary

vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'t': 0,
 'r': 1,
 'y': 2,
 'o': 3,
 'b': 4,
 'p': 5,
 'q': 6,
 'j': 7,
 'i': 8,
 'f': 9,
 "'": 10,
 'v': 11,
 'd': 12,
 'z': 13,
 'c': 14,
 'a': 15,
 'x': 16,
 'k': 17,
 'h': 18,
 'e': 19,
 ' ': 20,
 'n': 21,
 'u': 22,
 'm': 23,
 'g': 24,
 'l': 25,
 'w': 26,
 's': 27}

In [ ]:
# from the above tokens: we given the space token visibility by using the symbol (|)
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
# adding tokens for anything unknown discovered and padding for the blank token
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


Linear layer we add on top of the pretrained checkpoint will have an output dimension of 32.

In [ ]:
# jsonify the file next
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
# instantiate an object of the tokenizer class
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
# upload tokenizer to the Hugging Face Repo
# repo_name = "base-torgo"

In [ ]:
# push it to Hugging face to use it later
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/yip-i/wav2vec2-demo-F04-2/commit/a92a75a6c828e5fc85a1c98d3a2a12df55b5459e', commit_message='Upload tokenizer', commit_description='', oid='a92a75a6c828e5fc85a1c98d3a2a12df55b5459e', pr_url=None, pr_revision=None, pr_num=None)

# **Feature Extractor**

To convert speech to text: it has to first discretized: create individual units: called **sampling**

A higher sampling rate leads to a better approximation of the real speech signal but also necessitates more values per second

A Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, models should **always** make use of the `attention_mask` to mask padded tokens. However, due to a very specific design choice of `Wav2Vec2`'s "base" checkpoint, better results are achieved when using no `attention_mask`. 

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
# wrap the feature extractor and tokenizer into a single processor class: when testing will only need model and processor object
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

#**Prepare Dataset**

In [ ]:
torgo_dataset = torgo_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
torgo_dataset["train"][5]["audio"]

{'path': '/content/downloads/Torgo/F01/Session1/wav_arrayMic/0012.wav',
 'array': array([-0.00201416, -0.00595093, -0.00567627, ...,  0.0005188 ,
         0.00094604, -0.00186157], dtype=float32),
 'sampling_rate': 16000}

In [ ]:
# testing out sample audio files that have been loaded
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(torgo_dataset["train"]))

#print(torgo_dataset["train"][rand_int]["text"])
#ipd.Audio(data=np.asarray(torgo_dataset["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

# **Processing the dataset expected by the model**

1. load and resample the audio data: call batch["audio"]
2. extract values from the loaded audio file
3. encode the transcriptions to label ids

In [ ]:
def prepare_dataset(batch):
    # load the the audio data into batch
    audio = batch["audio"]

    # extract the values from the audio files
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    # encode it to the label ids
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
torgo_dataset = torgo_dataset.map(prepare_dataset, remove_columns=torgo_dataset.column_names["train"], num_proc=4)

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__`

Long input sequences require a lot of memory. Since `Wav2Vec2` is based on `self-attention` the memory requirement scales quadratically with the input length for long input sequences.

In [ ]:
torgo_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 4867
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 667
    })
})

### changed

In [ ]:
max_input_length_in_sec = 9.0
min_input_length_in_sec= 1.0
torgo_dataset["train"] = torgo_dataset["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
torgo_dataset["train"] = torgo_dataset["train"].filter(lambda x: x > min_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
torgo_dataset["test"] = torgo_dataset["test"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
torgo_dataset["test"] = torgo_dataset["test"].filter(lambda x: x > min_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])


  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
!git pull

fatal: not a git repository (or any of the parent directories): .git


# **Training and Evaluation**

**Need for a  data collabtor** <br>
wave2vec2 has a much larger input length as compared to the output length. For the input size, it is efficient to pad training batches to the longest sample in the batch (not overall sample)

In [ ]:
# data collator

import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

In [ ]:
# load the word error rate metric
wer_metric = load_metric("wer")


Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    print(pred_str)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### model assigning

In [ ]:
# assign the model
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    # "yip-i/uaspeech-pretrained",
    "facebook/wav2vec2-large-xlsr-53",
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)


# model: yongjian/wav2vec2-large-a contains [self-training] and has best wer of 0.557

Downloading:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'project_hid.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to u

The first component of wav2vec2 has acoustic features from the raw speech signal. This portion has been pretrained sufficiently and does not need to be pretrained anymore and hence freezed.

# **Define the parameters that are related to model training**


To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.

In [ ]:
model.freeze_feature_encoder()

In [ ]:
for name, param in model.named_parameters():
     print(name, param.requires_grad)

wav2vec2.masked_spec_embed True
wav2vec2.feature_extractor.conv_layers.0.conv.weight False
wav2vec2.feature_extractor.conv_layers.0.conv.bias False
wav2vec2.feature_extractor.conv_layers.0.layer_norm.weight False
wav2vec2.feature_extractor.conv_layers.0.layer_norm.bias False
wav2vec2.feature_extractor.conv_layers.1.conv.weight False
wav2vec2.feature_extractor.conv_layers.1.conv.bias False
wav2vec2.feature_extractor.conv_layers.1.layer_norm.weight False
wav2vec2.feature_extractor.conv_layers.1.layer_norm.bias False
wav2vec2.feature_extractor.conv_layers.2.conv.weight False
wav2vec2.feature_extractor.conv_layers.2.conv.bias False
wav2vec2.feature_extractor.conv_layers.2.layer_norm.weight False
wav2vec2.feature_extractor.conv_layers.2.layer_norm.bias False
wav2vec2.feature_extractor.conv_layers.3.conv.weight False
wav2vec2.feature_extractor.conv_layers.3.conv.bias False
wav2vec2.feature_extractor.conv_layers.3.layer_norm.weight False
wav2vec2.feature_extractor.conv_layers.3.layer_norm.bia

In [ ]:
# clear out cuda memory
import torch
torch.cuda.empty_cache()

### changed

In [ ]:
# parameters for the training
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir = repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=False,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
# pass all instances to the trainer as the final step before training
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=torgo_dataset["train"],
    eval_dataset=torgo_dataset["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:

trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4475
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 16800
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can pro

Step,Training Loss,Validation Loss,Wer
500,23.557600,3.365351,1.000000
1000,3.395300,3.172932,1.000000
1500,2.951400,2.894554,1.000000
2000,2.840000,2.838649,1.000000
2500,2.768500,2.714719,1.000000
3000,2.505900,2.134106,1.175182
3500,1.890700,1.360406,1.240268
4000,1.389200,0.881448,1.198905
4500,1.075400,0.641639,1.052920
5000,0.879500,0.575976,0.964112


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 655
  Batch size = 8


['t', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't', 't',

Saving model checkpoint to wav2vec2-demo-F04-2/checkpoint-500
Configuration saved in wav2vec2-demo-F04-2/checkpoint-500/config.json
Model weights saved in wav2vec2-demo-F04-2/checkpoint-500/pytorch_model.bin
Feature extractor saved in wav2vec2-demo-F04-2/checkpoint-500/preprocessor_config.json
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 655
  Batch size = 8


...

['fee t', 'yet he still thinks as swiftly as ever t', 'knew t', 'left  t', 'air t', 'torn        t', 'born     t', 'trait      t', 'stubble   t', 'you wished to know all about my grandfather t', 'chair  t', 'sticks       t', 'storm   t', 'rage     t', 'trace  t', 'go  t', 'beat t', 'air t', 'jacket   t', 'he dresed himself in an ancient black frock coat t', 'swarm      t', 'usualy minus several buttons t', 'store    t', 'knew t', 'chair  t', 'no   t', 'spark   t', 'grandfather likes to be modern ii in his language  nndt', 'tip   t', 'weed t', 'she had your dark suit in greasy wash water all year t', 'grow  t', 'two t', "don't askk me to cay an oily rag like that t", 'but he always answers bannana oil t', 'earor  t', 'bake t', 'well he is nearly ninetythree years old t', 'glow      t', 'kee t', 'park t', 'horn    t', 'goat t', 'dark t', 'bit    t', 'the t', 'gadget t', 'race t', 'raid   t', 'h slowhe slowly takes a short walk in the open air each day t', 'yes   t', 'giving thos ho obser

Saving model checkpoint to wav2vec2-demo-F04-2/checkpoint-16000
Configuration saved in wav2vec2-demo-F04-2/checkpoint-16000/config.json
Model weights saved in wav2vec2-demo-F04-2/checkpoint-16000/pytorch_model.bin
Feature extractor saved in wav2vec2-demo-F04-2/checkpoint-16000/preprocessor_config.json
Deleting older checkpoint [wav2vec2-demo-F04-2/checkpoint-15000] due to args.save_total_limit
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this mes

['fee t', 'yet he still thinks as swiftly as ever t', 'knew t', 'left  t', 'air t', 'torn     t', 'born  t', 'trait       t', 'stubble   t', 'you wished to know all about my grandfather t', 'chair  t', 'sticks      t', 'storm  t', 'rage    t', 'trace   t', 'go    t', 'beat t', 'air t', 'jacket   t', 'he dresed himself in an ancient black frock coat t', 'swarm        t', 'usualy minus several buttons t', 'store    t', 'knew t', 'chair   t', 'know      t', 'spark     t', 'grandfather likes to be modern ii in his language   nndt', 'tip  t', 'weed t', 'she had your dark suit in greasy wash water all year t', 'grow  t', 'two t', "don't askk me to cay an oily rag like that t", 'but he always answers bannana oil t', 'earor  t', 'bake t', 'well he is nearly ninetythree years old t', 'glow      t', 'kee t', 'park t', 'horn  t', 'goat t', 'dark t', 'bit   t', 'the t', 'gadget t', 'race t', 'raid    t', 'h slowhe slowly takes a short walk in the open air each day yt', 'yes     t', 'giving thos ho

Saving model checkpoint to wav2vec2-demo-F04-2/checkpoint-16500
Configuration saved in wav2vec2-demo-F04-2/checkpoint-16500/config.json
Model weights saved in wav2vec2-demo-F04-2/checkpoint-16500/pytorch_model.bin
Feature extractor saved in wav2vec2-demo-F04-2/checkpoint-16500/preprocessor_config.json
Deleting older checkpoint [wav2vec2-demo-F04-2/checkpoint-15500] due to args.save_total_limit
/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=16800, training_loss=1.5040524060385567, metrics={'train_runtime': 25185.5367, 'train_samples_per_second': 5.33, 'train_steps_per_second': 0.667, 'total_flos': 1.2483062591946273e+19, 'train_loss': 1.5040524060385567, 'epoch': 30.0})

### action of training

In [ ]:
!git init
%cd ..

Initialized empty Git repository in /content/downloads/.git/
/content


In [ ]:
# push to trained model to huggingface
trainer.push_to_hub("yip-i/" + repo_name)

Cloning https://huggingface.co/yip-i/wav2vec2-demo-F04-2 into local empty directory.


Download file pytorch_model.bin:   0%|          | 3.43k/1.18G [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.30k/3.30k [00:00<?, ?B/s]

Clean file training_args.bin:  30%|###       | 1.00k/3.30k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

Saving model checkpoint to wav2vec2-demo-F04-2
Configuration saved in wav2vec2-demo-F04-2/config.json
Model weights saved in wav2vec2-demo-F04-2/pytorch_model.bin
Feature extractor saved in wav2vec2-demo-F04-2/preprocessor_config.json


Upload file pytorch_model.bin:   0%|          | 3.30k/1.18G [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.30k/3.30k [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/yip-i/wav2vec2-demo-F04-2
   a92a75a..1df0b34  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/yip-i/wav2vec2-demo-F04-2
   a92a75a..1df0b34  main -> main

Dropping the following result as it does not have all the necessary fields:
{}
To https://huggingface.co/yip-i/wav2vec2-demo-F04-2
   1df0b34..c1a7570  main -> main

   1df0b34..c1a7570  main -> main



'https://huggingface.co/yip-i/wav2vec2-demo-F04-2/commit/1df0b348a6c7fe6e89a5c7c06cb77fee80c5b61c'

# **Evaluate**

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("yip-i/" + repo_name)

Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

loading configuration file preprocessor_config.json from cache at /root/.cache/huggingface/hub/models--yip-i--wav2vec2-demo-F04-2/snapshots/c1a75708b7c47592d21baabcc5cc11e9e87400ce/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}



Downloading:   0%|          | 0.00/236 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--yip-i--wav2vec2-demo-F04-2/snapshots/c1a75708b7c47592d21baabcc5cc11e9e87400ce/vocab.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--yip-i--wav2vec2-demo-F04-2/snapshots/c1a75708b7c47592d21baabcc5cc11e9e87400ce/tokenizer_config.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--yip-i--wav2vec2-demo-F04-2/snapshots/c1a75708b7c47592d21baabcc5cc11e9e87400ce/special_tokens_map.json
Adding <s> to the vocabulary
Adding </s> to the vocabulary
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# **Running experiments on own to learn about Wav2vec2 especially Transformers**

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# other libraries that are needed
import librosa
import torch 
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC

In [ ]:
# crucial role of the tokenizer: convert it into the input form
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/22aad52d435eb6dbaf354bdad9b0da84ce7d6156/vocab.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/22aad52d435eb6dbaf354bdad9b0da84ce7d6156/tokenizer_config.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/22aad52d435eb6dbaf354bdad9b0da84ce7d6156/special_tokens_map.json


Downloading:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/22aad52d435eb6dbaf354bdad9b0da84ce7d6156/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base-960h",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extrac

Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/22aad52d435eb6dbaf354bdad9b0da84ce7d6156/pytorch_model.bin
All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# previously loaded another audio file in .wav format
# now testing with torgo dataset file
audio_speech, rate = librosa.load('/content/downloads/Torgo/M03/Session2/wav_headMic/0191.wav', sr= 16000)
audio_speech

array([-0.01312256, -0.00540161, -0.01000977, ..., -0.00656128,
       -0.00872803,  0.00360107], dtype=float32)

In [ ]:
import IPython.display as display
display.Audio('/content/downloads/Torgo/M03/Session2/wav_headMic/0191.wav')

In [ ]:
# pass the audio to the tokenizer and get it back in pytorch format
input_values = tokenizer(audio_speech, return_tensors = 'pt')

# get the logits: non normalized predicted values
logits = model(input_values).logits

ValueError: ignored

In [ ]:
# store the logits

prediction = torch.argmax(logits)

In [ ]:
# decode the audio into text
transcription = tokenizer.decode(prediction)
#print(transcription)